In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/tr.zip
!unzip /content/drive/MyDrive/test.zip

Archive:  /content/drive/MyDrive/tr.zip
  inflating: train.csv               
Archive:  /content/drive/MyDrive/test.zip
   creating: test.csv/
  inflating: test.csv/test.csv       


In [ ]:
import pandas as pd
train = pd.read_csv('/content/train.csv')
submit = pd.read_csv('/content/test.csv/test.csv')

In [ ]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
    return x

import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text

In [ ]:
from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  


In [ ]:
train["comment_text"] = train["comment_text"].progress_apply(lambda x: clean_text(x))
train["comment_text"] = train["comment_text"].progress_apply(lambda x: clean_numbers(x))
train["comment_text"] = train["comment_text"].progress_apply(lambda x: replace_typical_misspell(x))
train['comment_text'] = train['comment_text'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))


  0%|          | 0/1804874 [00:00<?, ?it/s]

  0%|          | 0/1804874 [00:00<?, ?it/s]

  0%|          | 0/1804874 [00:00<?, ?it/s]

  0%|          | 0/1804874 [00:00<?, ?it/s]

In [ ]:
submit["comment_text"] = submit["comment_text"].progress_apply(lambda x: clean_text(x))
submit["comment_text"] = submit["comment_text"].progress_apply(lambda x: clean_numbers(x))
submit["comment_text"] = submit["comment_text"].progress_apply(lambda x: replace_typical_misspell(x))
submit['comment_text'] = submit['comment_text'].progress_apply(lambda x: clean_contractions(x, contraction_mapping))


  0%|          | 0/97320 [00:00<?, ?it/s]

  0%|          | 0/97320 [00:00<?, ?it/s]

  0%|          | 0/97320 [00:00<?, ?it/s]

  0%|          | 0/97320 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [ ]:
train['comment_text'] = train['comment_text'].astype(str) 

# List all identities
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian','transgender','heterosexual','bisexual','christian', 'jewish','muslim', 'black', 'white', 'psychiatric_or_mental_illness','hindu','buddhist','atheist','other_religion','asian','latino','other_race_or_ethnicity','intellectual_or_learning_disability']
TOXICITY_COLUMN = 'target' 

# Converting taget and identity columns to booleans
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, 1, 0)
    
def convert_dataframe_to_bool(df):
    bool_df = df.copy()
    for col in ['target'] + identity_columns:
        convert_to_bool(bool_df, col)
    return bool_df

train_embd = convert_dataframe_to_bool(train)    

In [ ]:
y = train_embd['target'].values
X = train_embd.drop('target',axis=1)

In [ ]:
del train_embd
del train

In [ ]:
 X_train, X_temp, y_train, y_temp = train_test_split(X,y,test_size=0.20,random_state=42)
 X_val, X_test, y_val, y_test = train_test_split(X_temp,y_temp,test_size=0.50,random_state=42)


In [ ]:
import numpy as np
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weights

array([0.54337358, 6.2638778 ])

In [ ]:
cw= dict()
for i,j in enumerate(class_weights):
    cw[i]=j

print(cw)  

{0: 0.5433735774019055, 1: 6.263877802457139}


In [ ]:
y_te = y_test 

In [ ]:
del X_temp
del y_temp

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train,num_classes=2)
y_test = tf.keras.utils.to_categorical(y_test,num_classes=2)
y_val = tf.keras.utils.to_categorical(y_val,num_classes=2)

In [ ]:
!pip install transformers


In [ ]:
import numpy as np
import pandas as pd
import transformers as tfm
from transformers import BertTokenizerFast



In [ ]:
model_class, tokenizer_class, pretrained_weights = (tfm.DistilBertModel,tfm.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
#model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [ ]:
del model_class
del tokenizer_class

In [ ]:

#train_tokenized = X_train['comment_text'].progress_apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
#test_tokenized = X_test['comment_text'].progress_apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
#val_tokenized = X_val['comment_text'].progress_apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

  0%|          | 0/1443899 [00:00<?, ?it/s]

In [ ]:
#def tokenizo(sentences, tokenizer):
#    m_len = 128
#    input_ids, input_masks, input_segments = np.zeros(shape=(len(sentences),m_len),dtype='int32'),np.zeros(shape=(len(sentences),m_len),dtype='int32'),np.zeros(shape=(len(sentences),m_len),dtype='int32')
#    count=0
#    for sentence in tqdm(sentences):
#        inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=m_len, padding='max_length', 
#                                             return_attention_mask=True, return_token_type_ids=True)
#        input_ids[count] = np.array(inputs['input_ids'],dtype='int32')
#        input_masks[count] = np.array(inputs['attention_mask'],dtype='int32')
#        input_segments[count] = np.array(inputs['token_type_ids'],dtype='int32')
#        count+=1
    
#    return input_ids,input_masks,input_segments

In [ ]:
train_input,train_attention_mask,train_segment = tokenizo(X_train['comment_text'], tokenizer)
del train_segment

In [ ]:
val_input,val_attention_mask,val_segment = tokenizo(X_val['comment_text'], tokenizer)
del val_segment

100%|██████████| 180487/180487 [04:54<00:00, 613.68it/s]


In [ ]:
test_input,test_attention_mask,test_segment = tokenizo(X_test['comment_text'], tokenizer)
del test_segment

100%|██████████| 180488/180488 [04:52<00:00, 617.10it/s]


In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
tr_sentences = X_train['comment_text'].values
train_input_ids = []
for sent in tr_sentences:
    encoded_sent = tokenizer.encode(sent,add_special_tokens = True)
    train_input_ids.append(encoded_sent)

MAX_LEN = 150
train_input_ids = pad_sequences(train_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")


In [ ]:
te_sentences = X_test['comment_text'].values
test_input_ids = []
for sent1 in te_sentences:
    encoded_sent1 = tokenizer.encode(sent1,add_special_tokens = True)
    test_input_ids.append(encoded_sent1)

MAX_LEN = 150
test_input_ids = pad_sequences(test_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")


In [ ]:
va_sentences = X_val['comment_text'].values
val_input_ids = []
for sent2 in va_sentences:
    encoded_sent2 = tokenizer.encode(sent2,add_special_tokens = True)
    val_input_ids.append(encoded_sent2)

MAX_LEN = 150
val_input_ids = pad_sequences(val_input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")


In [ ]:
train_attention_masks = []
for sent in train_input_ids:
    tr_att_mask = [int(token_id1 > 0) for token_id1 in sent]
    train_attention_masks.append(tr_att_mask)

In [ ]:
test_attention_masks = []
for sent1 in test_input_ids:
    te_att_mask = [int(token_id2 > 0) for token_id2 in sent1]
    test_attention_masks.append(te_att_mask)

In [ ]:
val_attention_masks = []
for sent2 in val_input_ids:
    val_att_mask = [int(token_id3 > 0) for token_id3 in sent2]
    val_attention_masks.append(val_att_mask)

In [ ]:
del tokenizer
del tqdm
del tf
del clean_text
del clean_numbers
del replace_typical_misspell
del  clean_contractions
del mispell_dict
del contraction_mapping

In [ ]:
del X_train

In [ ]:
train_inputs_ids = tf.convert_to_tensor(train_padded,dtype='int32')

In [ ]:
val_inputs_ids = tf.convert_to_tensor(val_padded,dtype='int32')

In [ ]:
test_inputs_ids = tf.convert_to_tensor(test_padded,dtype='int32')

In [ ]:
import numpy as np
train_attention_mask = tf.convert_to_tensor(np.where(train_padded != 0, 1, 0),dtype='int32')
train_attention_mask.shape

In [ ]:
val_attention_mask = tf.convert_to_tensor(np.where(val_padded != 0, 1, 0),dtype='int32')
val_attention_mask.shape

In [ ]:
test_attention_mask = tf.conver_to_tensor(np.where(test_padded != 0, 1, 0),dtype='int32')
test_attention_mask.shape

In [ ]:
import pickle
with open('/content/drive/MyDrive/train_inputs.pickle', 'wb') as f:
     pickle.dump(train_input_ids,f)

with open('/content/drive/MyDrive/y_train.pickle', 'wb') as f:
     pickle.dump(y_train,f)

with open('/content/drive/MyDrive/train_attention_mask.pickle', 'wb') as f:
     pickle.dump(train_attention_masks,f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/test_inputs.pickle', 'wb') as f:
     pickle.dump(test_input_ids,f)

with open('/content/drive/MyDrive/y_test.pickle', 'wb') as f:
     pickle.dump(y_test,f)

with open('/content/drive/MyDrive/test_attention_mask.pickle', 'wb') as f:
     pickle.dump(test_attention_masks,f)


with open('/content/drive/MyDrive/val_inputs.pickle', 'wb') as f:
     pickle.dump(val_input_ids,f)

with open('/content/drive/MyDrive/y_val.pickle', 'wb') as f:
     pickle.dump(y_val,f)

with open('/content/drive/MyDrive/val_attention_mask.pickle', 'wb') as f:
     pickle.dump(val_attention_masks,f)

In [ ]:
with open('/content/drive/MyDrive/test_df.pickle', 'wb') as f:
     pickle.dump(X_test,f)

In [ ]:
import pickle

file1 = open('/content/drive/MyDrive/train_inputs.pickle','rb')
train_input = pickle.load(file1)


file2 = open('/content/drive/MyDrive/test_inputs.pickle','rb')
test_input = pickle.load(file2)

file3 = open('/content/drive/MyDrive/val_inputs.pickle','rb')
val_input = pickle.load(file3)

file4 = open('/content/drive/MyDrive/train_attention_mask.pickle','rb')
train_attention_mask = pickle.load(file4)

file5 = open('/content/drive/MyDrive/test_attention_mask.pickle','rb')
test_attention_mask = pickle.load(file5)

file6 = open('/content/drive/MyDrive/val_attention_mask.pickle','rb')
val_attention_mask = pickle.load(file6)

file7 = open('/content/drive/MyDrive/y_train.pickle','rb')
y_train = pickle.load(file7)

file8 = open('/content/drive/MyDrive/y_val.pickle','rb')
y_val = pickle.load(file8)

file9 = open('/content/drive/MyDrive/y_test.pickle','rb')
y_test = pickle.load(file9)

In [ ]:
import pickle

file1 = open('/content/drive/MyDrive/train_inputs.pickle','rb')
train_inputs_ids = pickle.load(file1)

file4 = open('/content/drive/MyDrive/train_attention_mask.pickle','rb')
train_attention_mask = pickle.load(file4)

In [ ]:
#import tensorflow as tf
#train_attention_mask = tf.convert_to_tensor(train_attention_mask,dtype='int32')

ValueError: ignored

In [ ]:
import pickle
import tensorflow as tf

file1 = open('/content/drive/MyDrive/train_inputs.pickle','rb')
train_inputs_ids = tf.convert_to_tensor(pickle.load(file1))

file4 = open('/content/drive/MyDrive/train_attention_mask.pickle','rb')
train_attention_mask = tf.convert_to_tensor(pickle.load(file4))

file3 = open('/content/drive/MyDrive/val_inputs.pickle','rb')
val_input_ids = tf.convert_to_tensor(pickle.load(file3))

file6 = open('/content/drive/MyDrive/val_attention_mask.pickle','rb')
val_attention_mask = tf.convert_to_tensor(pickle.load(file6))

file7 = open('/content/drive/MyDrive/y_train.pickle','rb')
y_train = pickle.load(file7)

file8 = open('/content/drive/MyDrive/y_val.pickle','rb')
y_val = pickle.load(file8)

In [ ]:

file9 = open('/content/drive/MyDrive/test_inputs.pickle','rb')
test_inputs_ids = tf.convert_to_tensor(pickle.load(file9))

file11 = open('/content/drive/MyDrive/test_attention_mask.pickle','rb')
test_attention_mask = tf.convert_to_tensor(pickle.load(file11))

file12 = open('/content/drive/MyDrive/y_test.pickle','rb')
y_test = pickle.load(file12)

In [ ]:

file13 = open('/content/drive/MyDrive/test_df.pickle','rb')
X_test = pickle.load(file13)

In [ ]:
import numpy as np
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1804874, 368)

In [ ]:
del padded

In [ ]:
attention_mask = np.array(attention_mask,dtype='int32')

In [ ]:
import torch
attention_mask = torch.tensor(attention_mask)

In [ ]:
attention_mask.shape

torch.Size([1804874, 368])

In [ ]:
import pickle
with open('/content/drive/MyDrive/attention_mask.pickle', 'wb') as f:
     pickle.dump(attention_mask,f)

In [ ]:
import pickle
file = open('/content/drive/MyDrive/attention_mask.pickle','rb')
attention_mask = pickle.load(file)

file1 = open('/content/drive/MyDrive/attention_mask.pickle','rb')
attention_mask = pickle.load(file1)


In [ ]:
import tensorflow as tf

In [ ]:
input_ids = tf.convert_to_tensor(input_ids.numpy())

In [ ]:
attention_mask = tf.convert_to_tensor(attention_mask.numpy())

In [ ]:
print(attention_mask.shape)

torch.Size([1804874, 368])


In [ ]:
print(type(attention_mask))
print(type(input_ids))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 8.7 MB/s 
     |████████████████████████████████| 895 kB 64.6 MB/s 
     |████████████████████████████████| 50 kB 9.0 MB/s 
     |████████████████████████████████| 636 kB 83.2 MB/s 
     |████████████████████████████████| 3.3 MB 49.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers as tfm
model_class, tokenizer_class, pretrained_weights = (tfm.DistilBertModel,tfm.DistilBertTokenizer, 'distilbert-base-uncased')
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
del model

In [ ]:
del tokenizer_class
del model_class

In [ ]:
s0, s1, s2 = tf.split(x, num_or_size_splits=3, axis=1)


In [ ]:
 from transformers import  DistilBertConfig,TFDistilBertModel
import tensorflow as tf

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(50, return_sequences=True))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(50, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
  layer.trainable = False

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
from transformers import  DistilBertConfig,TFDistilBertModel
import tensorflow as tf
from tensorflow import keras

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(150,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(150,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(70, return_sequences=True))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(70, activation='relu',kernel_regularizer=keras.regularizers.l2(l2=0.1))(X)
X = tf.keras.layers.BatchNormalization()(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
  layer.trainable = False

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
from transformers import  DistilBertConfig,TFDistilBertModel
import tensorflow as tf
from tensorflow import keras

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(150,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(150,), name='masked_token', dtype='int32') 

embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(70, return_sequences=True,kernel_regularizer=keras.regularizers.l1_l2(l1=0.00001,l2=0.00001)))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(70, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(2, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)

for layer in model.layers[:3]:
    layer.trainable = False

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
# model_construction
from transformers import  DistilBertConfig,TFDistilBertModel
import tensorflow as tf
from tensorflow import keras

distil_bert = 'distilbert-base-uncased'
config = DistilBertConfig(dropout=0.0, attention_dropout=0.0)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(150,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(150,), name='masked_token', dtype='int32') 
embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
layer = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(150,return_sequences=True))(embedding_layer)
layer = tf.keras.layers.Dropout(0.5)(layer)
layer = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(70))(layer)
layer = tf.keras.layers.Dense(32)(layer)
layer = tf.keras.layers.Activation('sigmoid')(layer)
layer = tf.keras.layers.Dense(2,name='out_layer')(layer)
layer = tf.keras.layers.Activation('sigmoid')(layer)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = layer)

for layer in model.layers[:3]:
    layer.trainable = False


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

In [ ]:
#cw = {0: 0.5433735774019055, 1: 6.263877802457139}

In [1]:
#def warmup(epoch, lr):
#    """Used for increasing the learning rate slowly, this tends to achieve better convergence.
#    However, as we are finetuning for few epoch it's not crucial.
#    """
#    return max(lr +2e-4, 1e-3)

In [ ]:
def scheduler(epoch, lr):
    lr = lr*(0.68**epoch)
    return lr

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])

min delta=0.001 and changed to 0.0001

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4, min_delta=0.02, restore_best_weights=True),
    tf.keras.callbacks.LearningRateScheduler(scheduler, verbose=0),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=2e-5, patience=2, verbose=0, mode='auto', min_delta=0.001, cooldown=0, min_lr=2e-5)
]

In [ ]:
hist = model.fit(x=[train_inputs_ids,train_attention_mask],y=y_train, epochs=2,batch_size=512,validation_data=([val_input_ids,val_attention_mask],y_val),callbacks=callbacks)

Epoch 1/2
2821/2821 [==============================] - 4518s 2s/step - loss: 0.1479 - accuracy: 0.9448 - val_loss: 0.1361 - val_accuracy: 0.9483
Epoch 2/2
2821/2821 [==============================] - 4509s 2s/step - loss: 0.1286 - accuracy: 0.9506 - val_loss: 0.1290 - val_accuracy: 0.9500


In [ ]:
predict_val = model.predict([test_inputs_ids,test_attention_mask])

In [ ]:
predict_val

array([[1.5970103e-02, 9.8305571e-01],
       [7.7611440e-01, 2.2624317e-01],
       [9.9930823e-01, 6.2202505e-04],
       ...,
       [9.4805276e-01, 5.1538464e-02],
       [9.9940360e-01, 5.4881658e-04],
       [9.8807073e-01, 1.2215443e-02]], dtype=float32)

In [ ]:
SUBGROUP_AUC ='subgroup_auc'
BPSN_AUC ='bpsn_auc'
BNSP_AUC ='bnsp_auc'
def compute_bias_metrics_for_model(dataset,subgroups,model,label_col,include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model to get subgroup_auc ,bspn_auc & bpsn_auc"""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]==1])# so it picks the subgroup which are having toxic value(True)  eg subgroup='male' so we pick 'male'==True
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [ ]:
def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]==1] # that helps in judgeing the impact of particular positive identity column in given comment
    return compute_auc(subgroup_examples[label], subgroup_examples[model_name])

In [ ]:
def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan


In [ ]:
def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]==1) & ~(df[label]==1)]
    non_subgroup_positive_examples = df[~(df[subgroup]==1) & (df[label]==1)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[model_name])

In [ ]:
def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]==1) & (df[label]==1)]
    non_subgroup_negative_examples = df[~(df[subgroup]==1) & ~(df[label]==1)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[model_name])

In [ ]:
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    
#get_final_metric(bias_metrics_df, calculate_overall_auc(X_test_TFIDF, MODEL_NAME))

In [ ]:
#'intellectual_or_learning_disability'

In [ ]:
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian','transgender','heterosexual','bisexual','christian', 'jewish','muslim', 'black', 'white', 'psychiatric_or_mental_illness','hindu','buddhist','atheist','other_religion','asian','latino','other_race_or_ethnicity']
TOXICITY_COLUMN = 'target' 

In [ ]:
X_test['target']= y_te

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn import metrics
MODEL_NAME = 'bert_lstm'
X_test[MODEL_NAME] = list(predict_val[:,1]) # so we check for toxic comment 
bias_metrics_df = compute_bias_metrics_for_model(X_test, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
bias_metrics_df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
4,heterosexual,128,0.721649,0.835984,0.916122
2,homosexual_gay_or_lesbian,1116,0.805299,0.826671,0.954250
8,muslim,2101,0.824680,0.865804,0.945001
3,transgender,276,0.827192,0.876144,0.928926
10,white,2439,0.833045,0.813298,0.966434
9,black,1489,0.833202,0.810422,0.966118
5,bisexual,26,0.833333,0.848698,0.967803
17,latino,216,0.839481,0.876639,0.943330
18,other_race_or_ethnicity,55,0.865702,0.877803,0.954536
7,jewish,781,0.888618,0.898877,0.952625


In [ ]:
import numpy as np
bert_lstm_model=get_final_metric(bias_metrics_df, calculate_overall_auc(X_test,MODEL_NAME))
bert_lstm_model

0.9099356968167897

In [ ]:
model.save_weights("final_best_updated_model.h5")

# so in kaggle i used the saved weights of the model  and then  and i call the above architecture because kaggle is having rules that runtime of notebook while commiting should not be greater than 120

#and when i use  keras_model.h5 so we turning off the internet connection  when i will get all the necessary packages of transformer while commiting .Since  it will not accept the notebook becuase it wants the  internet connection should be off

refer : https://www.kaggle.com/ravikshdikola/notebook0a732ea8f3?scriptVersionId=73910199